## Setup

NOTE: All dependencies are within a conda environment to ensure reproducibility. To install all dependencies: pip install -r requirements.txt

In [ ]:
import tensorflow.compat.v1 as tf
#Lets see if tensorflow finds the GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# import tensorflow as tf

In [ ]:
#Lets see if it works
tf.ones(1) + tf.ones(1)

In [ ]:
import numpy as np # for working with arrays and matrices
import pandas as pd # for data manipulation and analysis
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for data visualization
import time # for time-related functions
import random # for random number generation
import cv2 # for computer vision and image processing tasks
import datetime # for saving date and time information

import csv


import h5py # for working with HDF5 (Hierarchical Data Format) files
import boto3 # for working with Amazon Web Services (AWS)
from pynwb import NWBHDF5IO # for working with Neurodata Without Border (NWB) files
import fsspec 
from fsspec.implementations.cached import CachingFileSystem # library used for working with various file systems in Python.
import requests 
import aiohttp # libraries which are used for making HTTP requests in Python.
import os # OS module provides various operating system-related functions to the code
# import csv # CSV module is used for working with CSV (Comma Separated Values) files in Python.
import pickle
import re


# used for splitting data into training and testing sets in Python.
from sklearn.model_selection import train_test_split 

# for generating a confusion matrix
from sklearn.metrics import confusion_matrix



# # Classes and functions from the Keras library which is used for building and training deep learning models in Python.
# from keras.models import load_model
# from keras.models import model_from_json
# from keras.utils import to_categorical
# from keras.models import Sequential
# from keras.layers import Dropout
# from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# These import the Adam optimizer class and various other classes from the TensorFlow Keras library 
# which is a high-level neural networks API used for building and training deep learning models in Python.
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
import imgaug.augmenters as iaa



# # Loading functions
from load_calcium_video import load_video_data
from pixel_values_normalization import normalize_video
from align_behavior_to_calcium import align_files
from class_balance import check_class_imbalance
from model_architecture import construct_model
from preprocessing_model import model_preprocessing
from run_model import model_execution
from save_model_info import save_training_info
# from set_s3_connection import generate_s3_url
from plots import plot_first_frames, plot_random_frames
from send_email_when_code_is_run import send_email
from class_balance import check_distribution_among_datasets


from urllib.parse import urlparse
from ann_visualizer.visualize import ann_viz
import importlib
import sys

import IPython
from IPython.display import display, Javascript


### Installing resnet-18

In [ ]:
# Goal: Document pipeline with turning labels, for 20 epochs, and save the results appropriately. Use seaborn
# Convert all plotting with seaborn
experiment_name = "Repeat comparison, between regular and base models, corridor location labels, 20% val - 80% train"
experiment_ID = '2.9'
%store experiment_ID
comment = str(experiment_name)
train_test_split_strategy = "20% val - 80% train"
name = 'BPNN_V3'
model_version = str(name)+'_1'
# experiment_version = ""

In [ ]:
# Define the name of the output directory
output_dir = "output"

# Check if the output directory already exists
if not os.path.exists(output_dir):
    # Create the output directory
    os.mkdir(output_dir)

    # Create the balance, accuracy, loss, and cm directories inside the output directory
    os.mkdir(os.path.join(output_dir, "balance"))
    os.mkdir(os.path.join(output_dir, "accuracy"))
    os.mkdir(os.path.join(output_dir, "loss"))
    os.mkdir(os.path.join(output_dir, "cm"))
    os.mkdir(os.path.join(output_dir, "architecture"))
    os.mkdir(os.path.join(output_dir, "pickles"))
else:
    print(f"The directory {output_dir} already exists.")

## [Start here]

In [ ]:
# videos = {
#     'animal3learningday9': model,
#     'animal3learningday10': tf,
#     'animal3learningday11': train_images
# }

In [ ]:
s3_calcium_url = 'https://s3.ki.se/dmc-striatum-arrowmaze/processed-data/miniscope-recordings/export-to-nwb/animal3learnday11/20211028_181307_animal3learnday11.nwb?AWSAccessKeyId=5AMYRX4EUZ0MV0276K24&Signature=3WXWM%2Ff3QBZ82lighTDw6fSonyM%3D&Expires=1681725833'
session_name = os.path.basename(os.path.dirname(urlparse(s3_calcium_url).path))
print(session_name) 

In [ ]:
video_data = load_video_data(s3_calcium_url)

#### Load all videos 

In [ ]:
# s3_calcium_url = ''
# name_animal3learnday9 = os.path.basename(os.path.dirname(urlparse(s3_calcium_url).path))
# print(name_animal3learnday9) 
# animal3learnday9 = load_video_data(s3_calcium_url)

In [ ]:
# s3_calcium_url = 'hattps://s3.ki.se/dmc-striatum-arrowmaze/processed-data/miniscope-recordings/export-to-nwb/animal3learnday11/20211028_181307_animal3learnday11.nwb?AWSAccessKeyId=5AMYRX4EUZ0MV0276K24&Signature=VAAKMB1%2FKndaAHOtraSaY3fKnV0%3D&Expires=1681472165'
# name_animal3learnday11 = os.path.basename(os.path.dirname(urlparse(s3_calcium_url).path))
# print(name_animal3learnday11) 
# name_animal3learnday11 = load_video_data(s3_calcium_url)

In [ ]:
# s3_calcium_url = ''
# name_animal3learnday10 = os.path.basename(os.path.dirname(urlparse(s3_calcium_url).path))
# print(name_animal3learnday10) 
# animal3learnday10 = load_video_data(s3_calcium_url)

In [ ]:
# s3_calcium_urls_dict = {
#     'animal3learnday9': 'https://s3.ki.se/dmc-striatum-arrowmaze/processed-data/miniscope-recordings/export-to-nwb/animal3learnday9/20211026_142935_animal3learnday9.nwb?AWSAccessKeyId=5AMYRX4EUZ0MV0276K24&Signature=GYmPHe6%2BzTK41rlQK6Pi9WwKtqk%3D&Expires=1681476565',
#     'animal3learnday10': 'https://s3.ki.se/dmc-striatum-arrowmaze/processed-data/miniscope-recordings/export-to-nwb/animal3learnday10/20211027_165052_animal3learnday10.nwb?AWSAccessKeyId=5AMYRX4EUZ0MV0276K24&Signature=4FaazGe6UnNxn10E9xH%2Bbivo61U%3D&Expires=1681476574',
#     'animal3learnday11': 'https://s3.ki.se/dmc-striatum-arrowmaze/processed-data/miniscope-recordings/export-to-nwb/animal3learnday11/20211028_181307_animal3learnday11.nwb?AWSAccessKeyId=5AMYRX4EUZ0MV0276K24&Signature=VAAKMB1%2FKndaAHOtraSaY3fKnV0%3D&Expires=1681472165'
# }

# # smth_name = [str(20211026_142935), str(20211027_165052),  str(20211028_181307)]

# # list of multiple urls
# # s3_calcium_urls = ['', '', 'https://s3.ki.se/dmc-striatum-arrowmaze/processed-data/miniscope-recordings/export-to-nwb/animal3learnday11/20211028_181307_animal3learnday11.nwb?AWSAccessKeyId=5AMYRX4EUZ0MV0276K24&Signature=VAAKMB1%2FKndaAHOtraSaY3fKnV0%3D&Expires=1681472165']
# video_data_list = []  # create empty list to store video data

# for name, url in s3_calcium_urls_dict.items():
#     # name_animal = os.path.basename(os.path.dirname(urlparse(s3_calcium_urls).path))
#     print(name) 
#     video_data = load_video_data(url)
#     video_data_list.append(video_data)  # append video data to the list

# printt(video_data_list)

In [ ]:
# csv_file = '/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/src/V2/aligned_videos_animal3.csv'  # path to your CSV file

# video_info = []  # create empty list to store video information

# with open(csv_file, newline='') as f:
#     reader = csv.reader(f)
#     for row in reader:
#         video_name = row[8]  # extract video name
#         crop_coords = tuple(map(int, row[1:]))  # extract crop coordinates and convert to tuple of integers
#         video_info.append((video_name, crop_coords))  # append video name and crop coordinates as a tuple to the list

# # Now video_info is a list of tuples, where each tuple contains the video name and its corresponding crop coordinates

In [ ]:
min_frame = np.min(video_data, axis=0)
video_data = video_data - min_frame

In [ ]:
images = normalize_video(video_data)

In [ ]:
# Check if the video has been loaded correctly and normalized
images[:1]

### Data Preparation

In [ ]:
# Determine the size of the calcium video dataset
num_of_frames = images.shape[0]
img_height = images.shape[1]
img_width = images.shape[2]
print("The number of video frames is ", num_of_frames, " and the frame dimensions (height x width) are: ", img_height, "X", img_width)

In [ ]:
# Preparing bonsai data file.
# CSV with additional data from the behavior box, such as reward deliveries. Also includes information needed for synchronizing the calcium and behavioral recordings.
bonsai_data = pd.read_csv('/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/tmaze_2021-10-28T18_13_23.csv', header=None)
# Segmentation of each frame into one behavior class.
df_behavior = pd.read_hdf('/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/20211028_181307_animal3learnday11.h5', 'per_frame')

In [ ]:
df_new_annotations, df_unique_states = align_files(bonsai_data, df_behavior)

In [ ]:
df_new_annotations_unique = df_new_annotations.unique()
df_new_annotations_unique

In [ ]:
save_dir = "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/src/V3/output/balance"

In [ ]:
class_counts, total_counts = check_class_imbalance(df_new_annotations, experiment_ID, save_dir)

### Data Verification

In [ ]:
# Training Parameters
channel_dimension = 1
labels = df_new_annotations
epochs = 20
batch_size = 32

In [ ]:
images.shape

We have 24186 images of dimensions 349x374 and the number 1 demonstrates that images are grayscale.

### Data Pre-processing

In [ ]:
# # Split the data into training and validation sets
split_index = int(0.2 * len(images))  # Index to split data
val_images, train_images = images[:split_index], images[split_index:]
val_labels, train_labels = labels[:split_index], labels[split_index:]

In [ ]:
check_distribution_among_datasets(val_labels, experiment_ID, save_dir, dataset_type = 'Validation_set')
check_distribution_among_datasets(train_labels, experiment_ID, save_dir, dataset_type = 'Training_set')

#### Test set?

In [ ]:
# Split the data into training and validation sets
# train_images, val_images, train_labels, val_labels = train_test_split(images.reshape(num_of_frames, img_height, img_width, channel_dimension), labels, test_size=0.2, random_state=42)

In [ ]:
train_images, val_images, train_labels, val_labels, num_classes = model_preprocessing(train_images, val_images, train_labels, val_labels, df_new_annotations_unique)

#### Subtract the minimum frame from all other frames

In [ ]:
# # min_frame = np.min(images, axis=0)
# # background_removed_frames = []
# # for frame in images:
# #     background_removed_frames.append(frame - min_frame)
    
# # images = np.array(background_removed_frames)
# # min_frame = np.min(train_images, axis=0)
# # train_images = train_images - min_frame

# min_frame_train = np.min(train_images, axis=0)
# train_images = train_images - min_frame_train

# min_frame_val = np.min(val_images, axis=0)
# val_images = val_images - min_frame_val

In [ ]:
# str(labels[2400])

In [ ]:
# sys.path.append('/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/src/V2/plots.py')

In [ ]:
# importlib.reload(plots.plot_first_frames)

In [ ]:
vmin = 0
vmax = 0.20

In [ ]:
# # Plot the first 5 random images
plot_first_frames(train_images, train_labels, vmin, vmax)
plot_first_frames(val_images, val_labels, vmin, vmax)

In [ ]:
plot_random_frames(train_images, train_labels, vmin, vmax)
plot_random_frames(val_images, val_labels, vmin, vmax)

#### Data Augmentation

In [ ]:
# # Perform data augmentation
# # Define the augmentation pipeline
# augmentation_pipeline = iaa.Sequential([
#     iaa.Fliplr(0.5), # flip horizontally with a probability of 0.5
#     iaa.Crop(percent=(0, 0.1)), # crop by up to 10% of the image width/height
#     iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.5))), # apply Gaussian blur with a probability of 0.5
#     # iaa.Affine(rotate=(-10, 10)) # rotate by up to 10 degrees
# ])

In [ ]:
# # Apply the augmentation pipeline to the training set
# augmented_train_images = []
# for image in train_images:
#     # Apply the same augmentation operation to both the image and its corresponding annotation
#     augmented_image = augmentation_pipeline(image=image)
#     augmented_train_images.append(augmented_image)


In [ ]:
# # Convert the augmented training set back to numpy arrays
# train_images = np.array(augmented_train_images)

In [ ]:
# plot_random_frames(train_images, labels)

### Build Model

In [ ]:
input_shape = (img_height, img_width, channel_dimension)

In [ ]:
# Create the model by calling the function
model = construct_model(input_shape, num_classes, name)

In [ ]:
# plot model architecture
plot_model(model, to_file='model_plot.png', show_shapes=False, show_layer_names=False)

In [ ]:
# import visualkeras
# from PIL import ImageFont
# visualkeras.layered_view(model, legend=True)
# # Define the directory name and the plot name
# dir_name = "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/src/V3/output/architecture"
# plot_name = f"{model_version}_architecture"
# ann_viz(model, view=True, filename=plot_name, title="CNN — "+str(name)+" — Simple Architecture")
# plot_path = os.path.join(dir_name, f"{plot_name}.png")
# plot = plt.gcf()
# plot.savefig(plot_path, dpi=300, bbox_inches="tight")

In [ ]:
# from ann_visualizer.visualize import ann_viz
# ann_viz(model, view=True, filename="cconstruct_model", title="CNN — Model 1 — Simple Architecture")

In [ ]:
# from keras.utils.vis_utils import plot_model
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True, rankdir='LR')

In [ ]:
validation_data=(val_images, val_labels)

### Train Model

In [ ]:
params = {
    'model': model,
    'tf': tf,
    'train_images': train_images,
    'train_labels': train_labels,
    'epochs': epochs,
    'batch_size': batch_size,
    'validation_data': validation_data,
    'val_images': validation_data[0],
    'val_labels': validation_data[1]
}

In [ ]:
save_dir = "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/src/V3/output/pickles"

In [ ]:
history = model_execution(params, save_dir, model_version)

In [ ]:
# # Call the send_email function after your model has finished training
# sender_email = 'guskikala@gmail.com'
# recipient_email = 'guskikala@gmail.com'
# subject = 'CNN Model Training Completed'
# message = 'Your CNN model training is complete!'

# send_email(sender_email, recipient_email, subject, message)

### Save results

In [ ]:
%store model
%store history
%store name
%store comment
%store experiment_ID
%store save_dir
%store model_version

In [ ]:
#save_training_info(model, history, video_name, comment, experiment_ID, save_dir, f1_score=f1_score)

High bias: If the training accuracy is low, it suggests that the model is underfitting the training data, i.e., it is not complex enough to capture the patterns in the data. In this case, you may need to increase the model's complexity by adding more layers or neurons, or by using a more complex architecture.

High variance: If the training accuracy is high but the validation accuracy is low, it suggests that the model is overfitting the training data, i.e., it is memorizing the training data instead of generalizing to new data. In this case, you may need to use regularization techniques like dropout or L2 regularization, or use early stopping to prevent the model from overfitting.

Good fit: If the training accuracy and validation accuracy are both high and close to each other, it suggests that the model is neither underfitting nor overfitting the data, i.e., it is generalizing well to new data.

Plateauing: If the validation accuracy is no longer increasing as the training set size or epochs increase, it suggests that the model has reached its capacity and adding more data or epochs is unlikely to improve its performance.

In general, a model accuracy curve can help you diagnose issues with your model and guide you in selecting appropriate strategies to improve its performance. It can also give you an idea of how much training data or how many epochs you need to achieve good performance.

In [ ]:
# def plot_confusion_matrix(model, images, labels, classes, title):
#     # Predict the class labels using the model
#     predicted_labels = np.argmax(model.predict(images), axis=1)

#     # Compute the confusion matrix using the predicted class labels and the true class labels
#     confusion = confusion_matrix(labels, predicted_labels)

#     # Plot the confusion matrix
#     fig, ax = plt.subplots(figsize=(10,10))
#     ax.imshow(confusion)
#     ax.set_xticks(np.arange(len(classes)))
#     ax.set_yticks(np.arange(len(classes)))
#     ax.set_xticklabels(classes)
#     ax.set_yticklabels(classes)
#     ax.set_xlabel('Predicted')
#     ax.set_ylabel('True')
#     ax.set_title(title)
#     plt.show()


In [ ]:

# # Plot the confusion matrix for the training set
# plot_confusion_matrix(model, train_images, train_labels, num_classes, 'Confusion Matrix for Training Set')

# # Plot the confusion matrix for the validation set
# plot_confusion_matrix(model, val_images, val_labels, num_classes, 'Confusion Matrix for Validation Set')

### Reflect on the results

1. Insufficient data? One calcium video of 24186 frames and with 349x374 dimensions.
2. Model architecture not appropriate. Try increasing the number of layers or filters, or adding more complex layers like BatchNormalization, Dropout, or Conv2DTranspose.
3. Incorrect data preprocessing
4. Incorrect hyperparameters
5. Class Imbalance (do oversampling, or undersampling)

### [Ignore for now]

In [ ]:
# from keras.layers import BatchNormalization
# from keras.preprocessing.image import ImageDataGenerator

# model = Sequential()

# model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(img_height, img_width, channel_dimension)))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())

# model.add(Dense(256, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Dense(128, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Dense(num_classes, activation='softmax'))

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # reshape train_images to have 4 dimensions
# train_images = np.expand_dims(train_images, axis=-1)

# # Reshape train_images to have 4 dimensions
# #train_images = np.squeeze(train_images)
# # train_images = np.squeeze(train_images, axis=-1)
# # train_images = np.squeeze(train_images, axis=-1)
# # train_images = np.squeeze(train_images, axis=-1)
# # train_images = np.expand_dims(train_images, axis=-1)


# # Data augmentation
# train_datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, 
#                                    shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest')

# history = model.fit(train_datagen.flow(train_images, train_labels, batch_size=batch_size),
#                     epochs=epochs,
#                     steps_per_epoch=len(train_images) // batch_size,
#                     shuffle=True)

In [ ]:
# Reusable snippets

In [ ]:
# Load calcium video from local environment
# with h5py.File('path', 'r') as f:
#     video_data = np.array(f['analysis/recording_20211016_163921-PP-BP-MC/data'])

In [ ]:
# Loading locally
# with h5py.File('/Users/konstantinoskalaitzidis/Developer/dmc/thesis_data/20211016_163921_animal1learnday1.h5', 'r') as f:
#     print(list(f.keys()))
#     behavior_data = np.array(f['per_frame'])

In [ ]:
# # save the model architecture to a JSON file
# with open('model_architecture.json', 'w') as f:
#     f.write(model.to_json())

In [ ]:
# # load the model architecture from the JSON file
# with open('model_architecture.json', 'r') as f:
#     json_string = f.read()

# model_json = model_from_json(json_string)

# # print the loaded model summary
# model.summary()

In [ ]:
# mySession = readSessionServer.SessionIterator()
# sess = mySession.findSession()
# # for sess in mySession.findSessions():
# #     print(sess)
# if sess.hasBehavior() and sess.hasCalcium():
#     behavior = sess.getBehaviorSegmentation(align_with_calcium=True).reset_index()

## [Ignore for now] Define hyperparameters

In [ ]:
# IMG_SIZE = 224
# BATCH_SIZE = 64
# EPOCHS = 10

# MAX_SEQ_LENGTH = 20
# NUM_FEATURES = 2048

In [ ]:
# import sys
# sys.path.append("/Users/konstantinoskalaitzidis/Developer/dmc")
# from readSessionsServer import SessionIterator

#TODO: Script to retrieve videos from a list of calcium videos (of the same animal) from the db

## Dataset preparation and label annotation (feature engineering)

### [Ignore for now] Data availale for processing - overview

The following is not going to be used for now but will allow us to have an overview of all the videos I have available to train my CNN model. I expect to have all recordings sessions for each animal as input for the CNN which is going to be trained only based on recordings from the corresponding animal. The data will be split to train/test at some point...

In [ ]:
# train_df = pd.read_csv("train.csv")
# test_df = pd.read_csv("test.csv")

# print(f"Total videos for training: {len(train_df)}")
# print(f"Total videos for testing: {len(test_df)}")

# train_df.sample(10)

Extract frames from the calcium imaging video and save to directory. Each frame contains spatial information, and the sequence of those frames contains temporal information (the latter is not exploited for now). Maybe also ask for path input from the user to make it reproducible for others.

Helpful source: https://keras.io/examples/vision/video_classification/

The number of frames may differ from video to video.
The frame rate may also differ from video to video but it should be 20fps for all. 

The duration of each frame depends on the frame rate of the video. If a video has a frame rate of 25 fps, then each frame will have a duration of 1/25th of a second, or approximately 0.04 seconds. The calcium videos use 20fps, while the behavioral recordings are at 60fps. Alignment of these videos will follow shortly. 

### [Ignore for now] Fetch all calcium videos from the dmc database and align calcium videos with behavior annotations

In [ ]:
# mySession = readSessionServer.SessionIterator()
# for sess in mySession.findSessions():
#     print(sess)
    # if sess.hasBehavior() and sess.hasCalcium():
        # behavior = sess.getBehaviorSegmentation(align_with_calcium=True).reset_index()

### [Ignore for now] Open calcium video locally, create dir for saving frames and count number of frames with OpenCV

In [ ]:
# Open the HDF5 file
# with h5py.File('/Users/konstantinoskalaitzidis/Developer/dmc/thesis_data/20211016_163921_animal1learnday1.nwb', 'r') as f:
#     # Print the keys of the file
#     print(list(f.keys()))
#     # dataset = f['identifier'][()]
#     # print(dataset)

In [ ]:
# Directory where frames from video will be stored after extraction
# frames_dir = "path"

In [ ]:
# Open the video using OpenCV and count the number of frames
# cap = cv2.VideoCapture(raw_calcium_video_path)
# frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# cap.release()

# print(f"Number of frames in the video: {frame_count}")

In [ ]:
# video = 'path'

# cap = cv2.VideoCapture(video)
# frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# cap.release()

# print(f"Number of frames in the video: {frame_count}")

In [ ]:
# Open the video file
# cap = cv2.VideoCapture(video)

# # Get the frame rate of the video
# frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

# # Release the video capture object
# cap.release()

# print(f"Frame rate of the video: {frame_rate}")

In [ ]:
# save each frame as one image

In [ ]:
# cap = cv2.VideoCapture(video)

# # Loop through the video frames and save each one as an image file
# frame_count = 0
# while(cap.isOpened()):
#     ret, frame = cap.read()
#     if ret == False:
#         break
#     # Save the frame as an image file
#     frame_file = os.path.join(frames_dir, "frame_" + str(frame_count) + ".jpg")
#     cv2.imwrite(frame_file, frame)
#     frame_count += 1

# # Close the video file
# cap.release()

In [ ]:
# # define paths
# video_path = '/Users/konstantinoskalaitzidis/Developer/dmc/thesis_data/20211016_163921_animal1learnday1.nwb'
# train_dir = '/Users/konstantinoskalaitzidis/Developer/dmc/thesis_data/train'
# test_dir = '/Users/konstantinoskalaitzidis/Developer/dmc/thesis_data/test'

# # define train-test split ratio
# train_test_ratio = 0.8

# # open video file
# cap = cv2.VideoCapture(video_path)

# # get video frame count
# frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# # create list of frame indices
# frame_indices = list(range(frame_count))

# # shuffle frame indices
# random.shuffle(frame_indices)

# # split frame indices into train and test sets
# train_frame_indices = frame_indices[:int(frame_count * train_test_ratio)]
# test_frame_indices = frame_indices[int(frame_count * train_test_ratio):]

# # iterate over frames and save to train or test directory
# for i in range(frame_count):
#     # read frame
#     ret, frame = cap.read()
#     if not ret:
#         break
    
#     # save frame to train or test directory
#     if i in train_frame_indices:
#         cv2.imwrite(os.path.join(train_dir, f'{i}.jpg'), frame)
#     else:
#         cv2.imwrite(os.path.join(test_dir, f'{i}.jpg'), frame)